# Tariff Calculator

In [172]:
import pdfplumber
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from tabulate import tabulate

# === TARIFF Model ===
def calculate_tariff(hs_code: str, origin: str) -> float:
    hs_code = hs_code.replace('.', '').strip()
    origin = origin.strip().upper()

    if hs_code.startswith('847141'):
        return 0.0
    elif hs_code.startswith('847330'):
        return 45.0 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('850440'):
        return 10.0 if origin == 'TAIWAN' else 55.0 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('854442'):
        return 12.6 if origin == 'TAIWAN' else 57.6 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('852910'):
        return 10.0 if origin == 'TAIWAN' else 30.0 if origin == 'CHINA' else 0.0
    elif hs_code.startswith('852589'):
        return 10.0 if origin == 'TAIWAN' else 55.0 if origin == 'CHINA' else 0.0
    return 0.0

# === BLOCK PARSER ===
def parse_item_block(block_lines):
    first_line = block_lines[0]
    item_name = None
    amount = None
    country = "NA"
    hs_code = "NA"
    htsus = "NA"

    match = re.search(r'POD-\d{6,7}[A-Z()]*\s+(.*?)\s+\d+\s+PCS\s+([\d,]+)', first_line)
    if match:
        raw_item = match.group(1).strip()
        item_name = re.sub(r'\s+\d+$', '', raw_item)
        amount = match.group(2).replace(",", "")
    else:
        return None

    for line in block_lines:
        if "Country of Origin" in line:
            origin_match = re.search(r"Country of Origin\s*[:：]?\s*([A-Z]+)", line, re.IGNORECASE)
            if origin_match:
                country = origin_match.group(1).upper()
        if "HS Code" in line:
            hs_match = re.search(r"HS Code\s*[:：]?\s*(\d+)", line)
            if hs_match:
                hs_code = hs_match.group(1)
        if "HTSUS" in line:
            htsus_match = re.search(r"HTSUS\s*[:：]?\s*([\d\.]+)", line)
            if htsus_match:
                htsus = htsus_match.group(1)

    tariff = calculate_tariff(hs_code, country)

    return {
        "Item": item_name,
        "Amount": amount,
        "Country of Origin": country,
        "HS Code": hs_code,
        "HTSUS": htsus,
        "Tariff (%)": tariff
    }

# === PDF PARSER ===
def extract_items_from_invoice(pdf_path):
    items = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            lines = page.extract_text().split('\n') if page.extract_text() else []
            current_block = []
            for line in lines:
                if 'PCS' in line:
                    if current_block:
                        item = parse_item_block(current_block)
                        if item:
                            items.append(item)
                    current_block = [line]
                else:
                    if current_block:
                        current_block.append(line)
            if current_block:
                item = parse_item_block(current_block)
                if item:
                    items.append(item)

    df = pd.DataFrame(items)
    if not df.empty:
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce').fillna(0)
        df['Tariff Amount'] = df['Amount'] * df['Tariff (%)'] / 100
        total_tariff = df['Tariff Amount'].sum()
        service_fee = df['Amount'].sum() * 0.003464
        total_fee = total_tariff + service_fee
        return df, total_tariff, service_fee, total_fee
    return df, 0.0, 0.0, 0.0

# === EXCEL WRITER ===
def write_tariff_to_excel(df, total_tariff, service_fee, total_fee, output_path):
    df.to_excel(output_path, index=False)

    wb = load_workbook(output_path)
    ws = wb.active

    start_row = ws.max_row + 2
    labels = ["TOTAL TARIFF", "MERCHANDISE SERVICE FEE", "TOTAL FEE"]
    values = [total_tariff, service_fee, total_fee]
    fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for i, (label, value) in enumerate(zip(labels, values)):
        row = start_row + i
        ws.cell(row=row, column=1, value=label).font = Font(bold=True)
        ws.cell(row=row, column=1).fill = fill
        cell = ws.cell(row=row, column=2, value=value)
        cell.font = Font(bold=True)
        cell.number_format = '"$"#,##0.00'
        cell.fill = fill

    wb.save(output_path)
    print(f"✅ Full tariff table with summary saved to: {output_path}")

# === USAGE ===
pdf_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Invoice\IN.PL_IN250804004_NTA.pdf"
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\06 Payment\APCC-W250700_Entry Summary_IN250_traiff excel.xlsx"

df_items, total_tariff, service_fee, total_fee = extract_items_from_invoice(pdf_path)
write_tariff_to_excel(df_items, total_tariff, service_fee, total_fee, output_path)
print(f'Total Items: {len(df_items)}')

print("\n" + "=" * 60)
print("\033[1;44m\033[1;37m     🚢  TARIFF CALCULATION SUMMARY  📦     \033[0m")
print("=" * 60)

print(f"\033[1;33m🔶 Total Tariff :\033[0m   \033[1m${total_tariff:,.2f}\033[0m")
print(f"\033[1;36m🔷 Service Fee  :\033[0m   \033[1m${service_fee:,.2f}\033[0m")
print(f"\033[1;32m🟩 Entry Fee    :\033[0m   \033[1m${total_fee:,.2f}\033[0m")

print("=" * 60 + "\n")

print("\033[1;34m📄 Tariff Breakdown Table:\033[0m")
print(tabulate(df_items, headers='keys', tablefmt='fancy_grid', showindex=False))


✅ Full tariff table with summary saved to: C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\06 Payment\APCC-W250700_Entry Summary_IN250_traiff excel.xlsx
Total Items: 48

     🚢  TARIFF CALCULATION SUMMARY  📦     
🔶 Total Tariff :   $2,482.83
🔷 Service Fee  :   $803.30
🟩 Entry Fee    :   $3,286.13

📄 Tariff Breakdown Table:
╒═════════════════════════════════════╤══════════╤═════════════════════╤═══════════╤════════════╤══════════════╤═════════════════╕
│ Item                                │   Amount │ Country of Origin   │   HS Code │ HTSUS      │   Tariff (%) │   Tariff Amount │
╞═════════════════════════════════════╪══════════╪═════════════════════╪═══════════╪════════════╪══════════════╪═════════════════╡
│ Nuvo-9002E-IFCN-CF2                 │    32220 │ TAIWAN              │    847150 │ 9903.01.32 │          0   │           0     │
├─────────────────────────────────────┼──────────┼─────────────────────┼───────────┼────────────┼──────────────┼─────────────────┤
│ PA-160W

# Packinglist Extraction

In [173]:
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        print(page.extract_text())

Invoice
Neousys Technology Inc.
11F., No.198, Jian 8th Rd.,
Zhonghe District, New Taipei City 235042
Taiwan
Phone:+886-2-22236182 FAX:+886-2-22236183
INV No.: IN250804004
INV Date: Aug 4, 2025.
Bill to : Ship to :
Neousys Technology America, Inc. Neousys Technology America, Inc.
55 East Hintz Road, Wheeling, IL 60090 55 East Hintz Road, Wheeling, IL 60090
United States United States
ATTN : Roy Wang ATTN : Roy Wang
Shipping Method: SPEEDMARK
Customer PO No. : POD-250705, POD-250726, POD-250727, POD-250850, POD-250865, POD- Currency : USD
250876, POD-250883, POD-250935, POD-250952, POD-250953, POD-250991, POD-251005, POD-
251007, POD-251008, POD-251013, POD-251020, POD-251026, POD-251030, POD-251031, POD-
251028, POD-251033, POD-251039, POD-251040, POD-251041, POD-251056, POD-251052, POD-
251063, POD-251071, POD-251076, POD-251093, POD-251092, POD-251091
Description of Industrial Computer or Computer
Item Cust. PO No. Unit Price QTY Unit Amount
accessory
1 POD-250705 Nuvo-9002E-IFCN-CF2 

In [174]:
import pdfplumber
import re
import pandas as pd

# === Pattern checkers ===
def is_box_header(line):
    return re.match(r"^\d+-\s*\d+", line)

def is_item_line(line):
    line = line.strip()
    if "TOTAL" in line.upper():
        return False
    return bool(re.search(r"\d+\s+\d+\.\d{2}\b", line))

def extract_box_number(line):
    match = re.match(r"^\d+-\s*\d+(?:-\d+)?", line)
    return match.group(0) if match else None

def extract_item_qty(line):
    # Remove Box No. prefix if present (e.g., "47- 2" or "47- 3-9")
    line_clean = re.sub(r'^\d+-\s*\d+(?:-\d+)?\s*', '', line)
    match = re.search(r'(?P<item>.+?)\s+(?P<qty>\d+)\s+\d+\.\d{2}', line_clean)
    if match:
        return match.group('item').strip(), int(match.group('qty'))
    return None, None

# === Parse PDF ===
invoice_lines = []
packing_lines = []
in_packing_list = False  # Switch flips at first box header

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        for line in page.extract_text().split('\n'):
            if not in_packing_list and is_box_header(line):
                in_packing_list = True
                packing_lines.append(line)
            elif in_packing_list and (is_box_header(line) or is_item_line(line)):
                packing_lines.append(line)
            elif not in_packing_list and is_item_line(line):
                invoice_lines.append(line)

# === Build DataFrames ===
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df_invoice = pd.DataFrame({"Invoice Lines": invoice_lines})

df_packing = pd.DataFrame({"Packing List Lines": packing_lines})
df_packing["Box No."] = df_packing["Packing List Lines"].apply(extract_box_number)
df_packing["Box No."] = df_packing["Box No."].ffill()

df_packing[["Product Number", "Qty"]] = df_packing["Packing List Lines"].apply(
    lambda line: pd.Series(extract_item_qty(line))
)
df_packing.drop(columns=['Packing List Lines'], inplace=True)

# === Output ===
print(f'Total Items:', len(df_packing))
print("\n📦 FULL PACKING LIST SECTION:")
print(df_packing)


Total Items: 54

📦 FULL PACKING LIST SECTION:
      Box No.                       Product Number  Qty
0     77- 1-6             SEMIL-2007-i9IC14-65W-DS    6
1    77- 7-13                  Nuvo-9002E-IFCN-CF2   28
2      77- 14                  Nuvo-9002E-IFCN-CF2    2
3      77- 15                           PA-160W-OW   20
4      77- 16                           PA-160W-OW   10
5   77- 17-21          Nuvo-7505D-i5HCG9-Tek02-100   20
6   77- 22-28                  Nuvo-9006E-IFCN-CF3   28
7      77- 29                  Nuvo-9006E-IFCN-CF3    2
8      77- 30                           Nuvo-9006E    2
9   77- 31-35         Nuvo-9002LP-i3QCGM-60W-SHI02   20
10     77- 36                          PA-280W-ET3   12
11     77- 37                     Cbl-PC-TW-180CM1   30
12     77- 37                            DINRAIL-O   55
13     77- 37      AccsyBx-Cardholder-10208GC-5080    1
14     77- 37              Cbl-M12A8M-2U2TA-180CM2    3
15     77- 37               NRU-230V-AWP Heat Sink    1
16

## Generate Receiving Log (Expend by Qty)

In [187]:
import pandas as pd

# List of prefixes to skip expansion
skip_prefixes = ("Cbl", "DIN", "AccsyBx", "Gpubr", "Ant", "FK", "TB")

# Add flag for whether to expand
df_packing["Expand"] = ~df_packing["Product Number"].str.startswith(skip_prefixes)

# Store original row order
df_packing["OriginalIndex"] = df_packing.index

# Build expanded list
expanded_rows = []

for _, row in df_packing.iterrows():
    if row["Expand"]:
        for _ in range(int(row["Qty"])):
            expanded_rows.append({
                "Box No.": row["Box No."],
                "Product Number": row["Product Number"],
                "Qty": pd.NA,  # use NA for blank (nullable int column)
                "OriginalIndex": row["OriginalIndex"]
            })
    else:
        expanded_rows.append({
            "Box No.": row["Box No."],
            "Product Number": row["Product Number"],
            "Qty": int(row["Qty"]),  # keep as integer
            "OriginalIndex": row["OriginalIndex"]
        })

# Create final DataFrame
final_df = pd.DataFrame(expanded_rows)

# Ensure Qty column is treated as nullable integer
final_df["Qty"] = final_df["Qty"].astype("Int64")

# Sort back to original order
final_df.sort_values("OriginalIndex", inplace=True)
final_df.drop(columns=["OriginalIndex", "Expand"], errors='ignore', inplace=True)
final_df.reset_index(drop=True, inplace=True)

# Reformat the columns to match your template
final_df["Box #"] = final_df["Box No."]
final_df["Part#"] = final_df["Product Number"]
final_df["POD#"] = ""   # Leave blank
final_df["SN#"] = ""    # Leave blank

# Reorder and select only the needed columns
output_df = final_df[["Box #", "POD#", "Part#", "SN#", "Qty"]]

# Save to Excel
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx"
output_df.to_excel(output_path, index=False)

# === Output ===
print(f'Total Items:', len(output_df))
print("\n📦 FULL PACKING LIST SECTION:")
print(output_df)



Total Items: 458

📦 FULL PACKING LIST SECTION:
         Box # POD#                                Part# SN#   Qty
0      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
1      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
2      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
3      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
4      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
5      77- 1-6                  SEMIL-2007-i9IC14-65W-DS      <NA>
6     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
7     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
8     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
9     77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
10    77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
11    77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
12    77- 7-13                       Nuvo-9002E-IFCN-CF2      <NA>
13    77- 7-13 

## Generate Incoming Form

In [170]:
import logging
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import Column, String, Float, Date, Integer


# Configure Flask app and SQLAlchemy
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:"
    "Czheyuan0227@aws-0-us-east-2.pooler.supabase.com:6543/postgres"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define the model
class ReceivingLog(db.Model):
    __tablename__ = 'receiving_log'
    id = Column(Integer, primary_key=True, autoincrement=True)
    serial_number = Column(String(255))
    entry_date    = db.Column(db.Date)
    invoice_number= db.Column(db.String(255))
    box_number    = db.Column(db.String(255))
    pod_number    = db.Column(db.String(255))
    part_number   = db.Column(db.String(255))
    quantity      = db.Column(db.INT)
    reference      = db.Column('Reference', db.Text) 

# Query the database inside the app context
with app.app_context():
    rows = ReceivingLog.query.filter_by(invoice_number='IN250728003').all()

    df = pd.DataFrame([{
        "Part Number": r.part_number,
        "Quantity": r.quantity,
        "Serial Number": r.serial_number,
        "Box Number": r.box_number,
        "POD Number": r.pod_number,
        
    } for r in rows])

    df['Serial Number'] = df['Serial Number'].fillna("NA").astype(str).str.strip()
    
# Step 1: Group and aggregate Quantity, preserving order
df_reset = df.reset_index()
grp = (
    df_reset
    .groupby(["Box Number", "Part Number", "POD Number"], as_index=False)
    .agg(
        Quantity=("Quantity", "sum"),
        first_idx=("index", "min")
    )
    .sort_values("first_idx")
    .drop(columns="first_idx")
    .reset_index(drop=True)
)

# Step 2: Build a serial number map per group (ignore exact duplicates, but keep 'NA')
serial_map = (
    df.groupby(["Box Number", "Part Number", "POD Number"])["Serial Number"]
    .apply(lambda x: ", ".join(x.astype(str).dropna().unique()))
    .reset_index()
    .rename(columns={"Serial Number": "Serial Numbers"})
)

# Step 3: Merge serial numbers into the main grouped result
grp_with_serials = grp.merge(
    serial_map,
    on=["Box Number", "Part Number", "POD Number"],
    how="left"
)

# Step 4: Reorder columns (optional)
grp_with_serials = grp_with_serials[
    ["Part Number", "Quantity", "Serial Numbers", "Box Number", "POD Number"]
]

print(f'Total Lines:', len(grp_with_serials))

# Step 5: Display in Jupyter notebook
from IPython.display import display, HTML
display(HTML(grp_with_serials.to_html(index=False)))



Total Lines: 68


Part Number,Quantity,Serial Numbers,Box Number,POD Number
Nuvo-9006DE-PoE,2,"P2900544, P2900545",59--1,POD-250596
Nuvo-9006DE-PoE-UL,24,"P2900519, P2900520, P2900521, P2900522, P2900523, P2900524, P2900525, P2900526, P2900527, P2900528, P2900529, P2900530, P2900531, P2900532, P2900533, P2900534, P2900535, P2900536, P2900537, P2900538, P2900539, P2900540, P2900541, P2900542",59--2--13,POD-250596
Nuvo-9006DE-PoE,10,"P2900556, P2900557, P2900558, P2900559, P2900560, P2900561, P2900562, P2900563, P2900564, P2900565",59--14--18,POD-250595
Nuvo-9006LP-PoE,8,"P2900546, P2900547, P2900548, P2900549, P2900550, P2900551, P2900552, P2900553",59--19--20,POD-250595
Nuvo-9006LP-PoE,2,"P2900554, P2900555",59--21,POD-250595
PA-280W-CW6P-2P,12,"P3000107, P3000108, P3000109, P3000110, P3000111, P3000112, P3000113, P3000114, P3000115, P3000116, P3000117, P3000118",59--22,POD-250954
NRU-230V-AWP-JAOi,2,"P3000034, P3000035",59--23--24,POD-250747
M.280-SSD-4TB-PCIe4-TLCWT5-IK,2,"Y5342506100030001, Y5342506100030002",59--23--24,POD-250747
AccsyBx-FP_M_C_K-NRU230VAWP-NTA,2,NA,59--23--24,POD-250747
GPSkit-mPCIe_GPS-M800,2,"P3000036, P3000037",59--23--24,POD-250747


In [171]:
from docx import Document

def write_table_to_word(df, doc_path, output_path):
    # Load the existing Word document
    doc = Document(doc_path)

    # Add a new table with headers + rows
    table = doc.add_table(rows=1, cols=len(df.columns))
    # table.style = 'Table Grid'

    # Write the header row
    hdr_cells = table.rows[0].cells
    for i, column_name in enumerate(df.columns):
        hdr_cells[i].text = column_name

    # Write the data rows
    for _, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)

    # Save to new Word file
    doc.save(output_path)
    print("✅ New table written to:", output_path)

# Example usage:
doc_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\Incoming Form Template.docx"
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\I07-070125-TaiwanHQ-IN240628006(Template).docx"
write_table_to_word(grp_with_serials, doc_path, output_path)



✅ New table written to: C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\I07-070125-TaiwanHQ-IN240628006(Template).docx
